In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

# Set random seed for reproducibility
torch.manual_seed(42)

In [2]:
# Define the RNN model
class SimpleRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleRNN, self).__init__()
        self.hidden_size = hidden_size
        self.rnn = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x, h0, c0):
        out, (hidden, cell) = self.rnn(x, (h0, c0))
        out = self.fc(out)  # Apply linear layer to all time steps
        return out, (hidden, cell)

# Hyperparameters
input_size = 1       # Each input is a single feature
hidden_size = 16     # Size of hidden state
output_size = 1      # Predicting a single value
seq_len = 10         # Length of each sequence
batch_size = 4       # Number of sequences in a batch
num_epochs = 200      # Number of training epochs
learning_rate = 0.01

In [3]:
# Create synthetic data
# Example: input sequences are sinusoids, and targets are shifted versions
x_train = torch.sin(torch.linspace(0, 2 * 3.1416, seq_len * batch_size).view(batch_size, seq_len, 1))
y_train = torch.cos(torch.linspace(0, 2 * 3.1416, seq_len * batch_size).view(batch_size, seq_len, 1))

# Initialize the model, loss function, and optimizer
model = SimpleRNN(input_size, hidden_size, output_size)
criterion = nn.MSELoss()  # Mean Squared Error Loss
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
in_data = torch.randn((batch_size, seq_len, input_size))
h0 = torch.zeros((1, batch_size, hidden_size))
c0 = torch.zeros((1, batch_size, hidden_size))
out, (hidden, cell) = model(in_data, h0, c0)

print(f'Output shape {out.shape}, hidden state shape: {hidden.shape}')

Output shape torch.Size([10, 1]), hidden state shape: torch.Size([1, 16])


In [18]:
# Training loop
for epoch in range(num_epochs):
    # Initialize the hidden state
    hidden = torch.zeros(1, batch_size, hidden_size)  # Shape: (num_layers, batch_size, hidden_size)
    cell = torch.zeros(1, batch_size, hidden_size)

    # Forward pass
    outputs, (hidden, cell) = model(x_train, hidden, cell)
    loss = criterion(outputs, y_train)  # Compute loss for all time steps
    
    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    # Print loss for every epoch
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

Epoch [1/200], Loss: 0.4682
Epoch [2/200], Loss: 0.4590
Epoch [3/200], Loss: 0.4501
Epoch [4/200], Loss: 0.4413
Epoch [5/200], Loss: 0.4321
Epoch [6/200], Loss: 0.4219
Epoch [7/200], Loss: 0.4102
Epoch [8/200], Loss: 0.3968
Epoch [9/200], Loss: 0.3827
Epoch [10/200], Loss: 0.3705
Epoch [11/200], Loss: 0.3622
Epoch [12/200], Loss: 0.3584
Epoch [13/200], Loss: 0.3576
Epoch [14/200], Loss: 0.3567
Epoch [15/200], Loss: 0.3538
Epoch [16/200], Loss: 0.3493
Epoch [17/200], Loss: 0.3443
Epoch [18/200], Loss: 0.3399
Epoch [19/200], Loss: 0.3365
Epoch [20/200], Loss: 0.3343
Epoch [21/200], Loss: 0.3331
Epoch [22/200], Loss: 0.3320
Epoch [23/200], Loss: 0.3304
Epoch [24/200], Loss: 0.3280
Epoch [25/200], Loss: 0.3250
Epoch [26/200], Loss: 0.3221
Epoch [27/200], Loss: 0.3196
Epoch [28/200], Loss: 0.3179
Epoch [29/200], Loss: 0.3168
Epoch [30/200], Loss: 0.3160
Epoch [31/200], Loss: 0.3151
Epoch [32/200], Loss: 0.3139
Epoch [33/200], Loss: 0.3125
Epoch [34/200], Loss: 0.3109
Epoch [35/200], Loss: 0

In [19]:
# Test the model (inference)
with torch.no_grad():
    test_hidden = torch.zeros(1, batch_size, hidden_size)
    test_cell = torch.zeros(1, batch_size, hidden_size)
    test_outputs, _ = model(x_train, test_hidden, test_cell)
    print("\nSample output (first sequence):")
    print("Predicted:", test_outputs[0].squeeze().numpy())
    print("Target:   ", y_train[0].squeeze().numpy())


Sample output (first sequence):
Predicted: [0.0549446  0.5168408  1.073276   0.8731556  0.79805285 0.69636005
 0.55725634 0.42824063 0.28370196 0.12557617]
Target:    [1.         0.9870502  0.9485362  0.8854555  0.7994419  0.69272304
 0.5680629  0.42869017 0.2782146  0.12053337]


In [ ]:
# device = torch.device('cpu')
x = torch.zeros(100, 2, dtype=torch.float32, device=device)
x.device

TypeError: 'module' object is not callable